
<br>
Transform Storm Tracker L0 data to L1 SHARPpy format file.<br>
Hungjui Yu<br>
20200504<br>


In [1]:
import pandas as pd
from datetime import datetime 
import pytz
# import metpy.calc as mpcalc
from metpy.calc import dewpoint_from_relative_humidity
from metpy.units import units

In [46]:
# Set ST node number:

ST_no = 2968

# Set ST launch time (UTC):

launch_time_from_log = '20210503184852'

# Specified timezones:

# pytz.all_timezones
tz_utc = pytz.timezone('UTC')
tz_fc = pytz.timezone('US/Mountain')

# Specified the launch time in UTC:

launch_time = datetime.strptime(launch_time_from_log, '%Y%m%d%H%M%S')
launch_time_utc = tz_utc.localize(launch_time)

# print(launch_time_utc)
# print(launch_time_from_log[:8])

# Load raw data:
L0_raw_data = pd.read_csv('../L0/' + launch_time_from_log[:8] + '/no_{}.csv'.format(ST_no))

# L0_raw_data.head()

# Convert the data time to datetime object:

# print(L0_raw_data['Time'])
# print(L0_raw_data.info())

L0_raw_data['Time'] = pd.to_datetime(L0_raw_data['Time'], utc=tz_utc)

# print(L0_raw_data.info())
print(L0_raw_data['Time'][0])

# Calculate dew-point temperature in raw data:

L0_raw_data['dT(degC)'] = dewpoint_from_relative_humidity((L0_raw_data['Temperature(degree C)'].to_numpy() * units.degC).to(units.K), L0_raw_data['Humidity(%)'].to_numpy() / 100.)

# print(L0_raw_data['dT(degC)'])

# Convert wind speed in raw data:

L0_raw_data['WS(kts)'] = (L0_raw_data['Speed(km/hr)'].to_numpy() * units.kilometer_per_hour).to(units.knot)

# print(L0_raw_data['WS(kts)'], L0_raw_data['Speed(km/hr)'])

# Convert wind direction in raw data:

L0_raw_data.loc[L0_raw_data['Direction(degree)'] <= 180, 'WDIR'] = L0_raw_data['Direction(degree)'] + 180
L0_raw_data.loc[L0_raw_data['Direction(degree)'] > 180, 'WDIR'] = L0_raw_data['Direction(degree)'] - 180
L0_raw_data.loc[L0_raw_data['Speed(km/hr)'] == 0, 'WDIR'] = 0

# print(L0_raw_data['Direction(degree)'])
# print(L0_raw_data['WDIR'])

# Find the launch time in ST Time:

# L0_data = L0_raw_data[['Pressure(hPa)', 'Temperature(degree C)']]
L1_data_sharppy = L0_raw_data[L0_raw_data['Time'] >= launch_time_utc][ \
                                                                      ['Pressure(hPa)', \
                                                                       'Height(m)', \
                                                                       'Temperature(degree C)', \
                                                                       'dT(degC)', \
                                                                       'WDIR', \
                                                                       'WS(kts)'] \
                                                                     ]

print(L1_data_sharppy)

# Output L1 data (ascii format):

# L1_data_sharppy.to_csv("L1_test.csv", header=["LEVEL", "HGHT", "TEMP", "DWPT", "WDIR", "WSPD"], index=False)

L1_sharppy_filename = 'no_{}_L1_sharppy.txt'.format(ST_no)

with open(L1_sharppy_filename, 'w') as file:
    
    file.write('%TITLE%\n')
    file.write(' CSU-PRECIP-ST-{}'.format(ST_no) + ' ' + launch_time_from_log[2:8] + '/' + launch_time_from_log[8:12] + '\n')
    file.write('\n')
    file.write('   LEVEL       HGHT       TEMP       DWPT       WDIR       WSPD\n')
    file.write('-------------------------------------------------------------------\n')
    file.write('%RAW%\n')

L1_data_sharppy.to_csv(L1_sharppy_filename, mode='a', header=False, index=False)
    
with open(L1_sharppy_filename, 'a') as file:
      
    file.write('%END%')

2021-05-03 18:34:02+00:00
      Pressure(hPa)  Height(m)  Temperature(degree C)   dT(degC)    WDIR  \
887      841.333281    1572.82                   7.56   3.640334  225.64   
888      841.334609    1573.07                   7.56   3.677550  225.64   
889      841.175156    1574.76                   7.55   3.667857  225.64   
890      840.801875    1577.50                   7.52   3.564283  309.20   
891      840.384140    1580.70                   7.49   3.460397  330.17   
...             ...        ...                    ...        ...     ...   
3605     310.205078    9032.52                 -29.00 -51.260915  239.87   
3606     310.113320    9035.18                 -29.02 -51.189754  243.31   
3607     308.563828    9067.61                 -29.23 -51.445674  238.40   
3608     308.434921    9071.93                 -29.28 -51.485844  238.18   
3609     306.859101    9100.87                 -29.65 -52.048389  251.94   

        WS(kts)  
887    0.107991  
888    0.334773  
889    